# Les «f-strings» revisitées

Les «chaînes formatées» sont des chaînes préfixées par la lettre `f`.

À l'intérieur d'une telle chaîne, les partie entre accolades `{...}` peuvent contenir des expressions dont le résultat est inséré dans la chaîne finale.

Basiquement:

In [ ]:
prenom, nom, age = "john", "smith", 31
f_str = f"Je m'appelle {prenom} {nom.upper()} et j'ai {age} ans"
print(f_str)

Pour une variable `nb` de type `int`, on peut formatter sa représentation en *binaire* `{nb:b}`, en *octale* `{nb:o}`, ou en *hexadécimal* `{nb:x}`:

In [ ]:
nb = 76 # essayer avec 0x1001100, 0o114 et 0x4C
print(f"{nb} s'écrit {nb:b} en base 2, {nb:o} en base 8 et {nb:x} en base 16")

Notamment, il peut être utile de calibrer l'écriture en binaire sur 1 octet `{nb:08b}` ou sur 2 octets `{nb:016b}`.

Le `'0'` après les `:` indique le caractère de remplissage à gauche et `8` ou `16` indique le nombre de «caractères» utilisés pour écrire le nombre:

In [ ]:
nb1, nb2 = 76, 345
print(f"Le premier nombre est {nb1:08b} et le second {nb2:016b}")

**Alternativement**, on peut préciser qu'on souhaite séparer les groupes de 4 bits par `_`

In [ ]:
nb1, nb2 = 76, 345
print(f"Le premier nombre est {nb1:_b} et le second {nb2:_b}")

C'est similaire avec les nombres en base 10 (flottants ou entier):

In [ ]:
nb1, nb2 = 10000000000000, 1.2e10
print(f"{nb1:_} et {nb2:_}")

# Le module `struct`

La fonction `pack(format, valeur)` convertie la `valeur` Python en un type du langage C précisé par `format`; la valeur renvoyée est un **tableau d'octets** (*bytesarray*).

`format` est une chaîne de caractère:
- `h` ou `H`: vers le type *signed short* ou *unsigned short* (entier sur 2 octets)
- `i` ou `I`: vers le type *signed int* ou *unsigned int* (entier sur 4 octets)
- `f` ou `d`: vers le type *float* (flottant en simple précision) ou *double* (double précision).
- ...

In [ ]:
from struct import pack
x = 12 # 0000 0000 0000 1100 (2 octets) 
octets = pack('h', x)
print(octets)

**Explication**: la machine est petit boutiste - *little endian* - donc le premier octet est celui de droite `0c` et le second est celui de gauche `00`!

Observer que Python représente l'octet dans le tableau avec l'écriture `\x--`.

**Cependant**, on peut forcer la représentation «gros boutiste» - *big endian* - en utilisant le format `'!h'`

In [ ]:
octets = pack('!h', x)
print(octets)

Trouvons la représentation binaire du nombre:

In [ ]:
octets_b = [ f"{octet:08b}" for octet in octets ]
" ".join(octets_b)

Essayons avec un nombre négatif:

In [ ]:
x = -12
octets = pack('!h', x) # essayer de remplacer !h par !H
octets_b = [ f"{octet:08b}" for octet in octets ]
" ".join(octets_b)

Il n'est pas trop difficile de voir que c'est la représentation en complément à 2 sur deux octets du nombre $-12$.

Vérifions qu'on retrouve la représentation binaire du flottant $-1,\!375\times 2^{-5}$

In [ ]:
x = -1.375 * 2 ** (-5)
# récupérons le tableau d'octets gros-boutiste de x vu comme un flottant en simple précision 
octets = pack('!f', x)
# conversion de chaque octet en son motif binaire
octets_b = [ f"{octet:08b}" for octet in octets]
# collons les bouts
repr_bin = "".join(octets_b)
# 1er bit (n° 0) = signe, bit 1 à 9 = exposant, bit 10 à 31 = mantisse. 
signe, exposant, mantisse = repr_bin[0], repr_bin[1:9], repr_bin[9:]
# affichons le tout proprement
print(f"{signe} {exposant} {mantisse}")

### Note complémentaire

Lorsqu'on affiche directement le tableau d'octets, Python écrit sois:
- `\x--`: pour représenter un octet qui **n'est pas** le *point de code* ASCII d'un caractère imprimable,
- autrement le caractère ASCII lui-même!.

Cela donne parfois quelquechose de peu lisible:

In [ ]:
x = -1.375 * 2 ** (-5)
# récupérons le tableau d'octets gros-boutiste de x vu comme un flottant en simple précision 
octets = pack('!f', x)
print(octets, len(octets))

Il y a bien 4 octets, le deuxième est 0011 0000 soit «0x30» (hexa) qui vaut 32+16=48; c'est le point de code du caractère '0'!

Il faut lire «bd 30 00 00» en hexadecimal, la preuve:

In [ ]:
octets_hex = [ f"{octet:02x}" for octet in octets ]
print(" ".join(octets_hex))